In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import os
import warnings
import sys
from glob import glob
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import StackingRegressor, VotingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor

import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

from category_encoders import TargetEncoder

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Define file paths (Update these paths accordingly)
TRAIN_DATA_DIR = './TrainingData'  # Replace with your training data directory path
SUBMISSION_PATH = './submission.csv'           # Replace with your desired submission path

# Define constants
TARGET_COLUMN = 'Power(mW)'  # Replace with your target column name
ID_COLUMN = 'LocationCode'                     # Replace with your ID column name (if available)
N_SPLITS = 5                         # Number of folds for cross-validation
RANDOM_STATE = 42                    # Random seed for reproducibility

# Function to load and concatenate multiple CSV files
def load_and_concatenate_csvs(data_dir):
    """
    Load all CSV files from the specified directory and concatenate them into a single DataFrame.
    
    Parameters:
        data_dir (str): Path to the directory containing CSV files.
    
    Returns:
        pd.DataFrame: Concatenated DataFrame containing all training data.
    """
    all_files = glob(os.path.join(data_dir, "*.csv"))
    if not all_files:
        print(f"No CSV files found in directory: {data_dir}")
        sys.exit(1)
    
    list_df = []
    for file in all_files:
        try:
            df = pd.read_csv(file)
            list_df.append(df)
            print(f"Loaded file: {file} with shape {df.shape}")
        except Exception as e:
            print(f"Error loading {file}: {e}")
    
    combined_df = pd.concat(list_df, ignore_index=True)
    print(f"Combined DataFrame shape: {combined_df.shape}")
    return combined_df

# Function to preprocess data
def preprocess_data(df):
    """
    Preprocess the combined DataFrame.
    - Handle missing values
    - Encode categorical variables appropriately
    - Feature scaling
    - Split into training and validation sets
    
    Parameters:
        df (pd.DataFrame): Combined DataFrame containing all training data.
    
    Returns:
        X_train (pd.DataFrame): Preprocessed training features.
        X_val (pd.DataFrame): Preprocessed validation features.
        y_train (pd.Series): Training target.
        y_val (pd.Series): Validation target.
        ids_train (pd.Series): Training IDs (if available).
        ids_val (pd.Series): Validation IDs (if available).
        scaler (StandardScaler): Fitted scaler object.
        encoder (TargetEncoder): Fitted encoder object.
    """
    # Identify if ID_COLUMN exists and separate it
    if ID_COLUMN in df.columns:
        ids = df[ID_COLUMN]
        df = df.drop(columns=[ID_COLUMN])
    else:
        ids = None

    # Separate features and target
    X = df.drop(columns=[TARGET_COLUMN])
    y = df[TARGET_COLUMN]

    # Identify numerical and categorical columns
    numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
    categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()

    # Handle missing values
    for col in numerical_cols:
        median = X[col].median()
        X[col].fillna(median, inplace=True)

    for col in categorical_cols:
        mode = X[col].mode()[0]
        X[col].fillna(mode, inplace=True)

    # Identify high and low cardinality categorical columns
    high_cardinality_cols = [col for col in categorical_cols if X[col].nunique() > 100]
    low_cardinality_cols = [col for col in categorical_cols if X[col].nunique() <= 100]

    print(f"High-Cardinality Columns ({len(high_cardinality_cols)}): {high_cardinality_cols}")
    print(f"Low-Cardinality Columns ({len(low_cardinality_cols)}): {low_cardinality_cols}")

    # Handle low-cardinality categorical columns with one-hot encoding
    X_encoded = pd.get_dummies(X, columns=low_cardinality_cols, drop_first=True)
    print(f"Shape after one-hot encoding low-cardinality columns: {X_encoded.shape}")

    # Handle high-cardinality categorical columns with target encoding
    if high_cardinality_cols:
        encoder = TargetEncoder(cols=high_cardinality_cols)
        X_encoded[high_cardinality_cols] = encoder.fit_transform(X_encoded[high_cardinality_cols], y)
        print(f"Applied target encoding on high-cardinality columns.")
    else:
        encoder = None

    # Feature Scaling
    scaler = StandardScaler()
    X_encoded[numerical_cols] = scaler.fit_transform(X_encoded[numerical_cols])

    # Optional: Feature Engineering can be added here

    # Split into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(
        X_encoded, y, test_size=0.2, random_state=RANDOM_STATE
    )

    # If IDs were present, split them accordingly
    if ids is not None:
        ids_train, ids_val = train_test_split(
            ids, test_size=0.2, random_state=RANDOM_STATE
        )
    else:
        ids_train, ids_val = None, None

    print(f"Training set shape: {X_train.shape}")
    print(f"Validation set shape: {X_val.shape}")

    return X_train, X_val, y_train, y_val, ids_train, ids_val, scaler, encoder

# Function to define individual models
def define_models():
    """
    Define all individual models to be compared.
    
    Returns:
        dict: Dictionary of model names and their corresponding instances.
    """
    models = {}
    
    # LightGBM Regressor
    models['LightGBM'] = lgb.LGBMRegressor(
        objective='regression',
        num_leaves=31,
        learning_rate=0.05,
        n_estimators=1000,
        random_state=RANDOM_STATE
    )
    
    # XGBoost Regressor
    models['XGBoost'] = xgb.XGBRegressor(
        objective='reg:squarederror',
        n_estimators=1000,
        learning_rate=0.05,
        max_depth=7,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=RANDOM_STATE,
        n_jobs=-1
    )
    
    # CatBoost Regressor
    models['CatBoost'] = CatBoostRegressor(
        iterations=1000,
        learning_rate=0.05,
        depth=6,
        loss_function='MAE',
        eval_metric='MAE',
        random_seed=RANDOM_STATE,
        early_stopping_rounds=50,
        verbose=0
    )
    
    # Define a simpler and optimized MLPRegressor
    models['MLPRegressor'] = MLPRegressor(
        hidden_layer_sizes=(64, 32),  # Reduced complexity
        activation='relu',
        solver='adam',
        alpha=0.0001,
        batch_size='auto',
        learning_rate='adaptive',
        max_iter=200,  # Limited iterations
        tol=1e-3,       # Increased tolerance
        early_stopping=True,  # Enable early stopping
        validation_fraction=0.1,
        random_state=RANDOM_STATE,
        verbose=True  # Enable verbose to monitor progress
    )
    
    return models

# Function to define ensemble models
def define_ensemble_models():
    """
    Define Stacking and Voting Ensemble models.
    
    Returns:
        dict: Dictionary of ensemble model names and their corresponding instances.
    """
    # Define base learners
    base_learners = [
        ('lgb', lgb.LGBMRegressor(n_estimators=500, learning_rate=0.05, random_state=RANDOM_STATE)),
        ('xgb', xgb.XGBRegressor(n_estimators=500, learning_rate=0.05, random_state=RANDOM_STATE)),
        ('cat', CatBoostRegressor(iterations=500, learning_rate=0.05, depth=6, loss_function='MAE',
                                 eval_metric='MAE', random_seed=RANDOM_STATE, verbose=0))
    ]
    
    # Define Stacking Regressor with Linear Regression as meta-learner
    stacking_regressor = StackingRegressor(
        estimators=base_learners,
        final_estimator=LinearRegression(),
        passthrough=True,
        cv=5,
        n_jobs=-1
    )
    
    # Define Voting Regressor (soft voting via averaging)
    voting_regressor = VotingRegressor(
        estimators=base_learners,
        weights=[1, 1, 1]
    )
    
    ensemble_models = {
        'StackingRegressor': stacking_regressor,
        'VotingRegressor': voting_regressor
    }
    
    return ensemble_models

# Function to define Neural Network using TensorFlow Keras
def define_keras_model(input_dim):
    """
    Define and compile a Keras Neural Network model.
    
    Parameters:
        input_dim (int): Number of input features.
    
    Returns:
        tf.keras.Model: Compiled Keras model.
    """
    model = Sequential()
    model.add(Dense(128, activation='relu', input_dim=input_dim))
    model.add(Dropout(0.3))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1))  # Output layer for regression
    model.compile(optimizer='adam', loss='mae')
    return model

# Function to evaluate models using K-Fold Cross-Validation
def evaluate_models(models, X_train, y_train):
    """
    Evaluate each model using K-Fold Cross-Validation and return MAE scores.
    
    Parameters:
        models (dict): Dictionary of model names and instances.
        X_train (pd.DataFrame): Training features.
        y_train (pd.Series): Training target.
    
    Returns:
        dict: Dictionary of model names and their average MAE scores.
    """
    kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)
    mae_scores = {}
    
    for name, model in models.items():
        print(f"Training and evaluating model: {name}")
        fold_mae = []
        
        for fold, (train_idx, val_idx) in enumerate(kf.split(X_train)):
            X_tr, X_fold = X_train.iloc[train_idx], X_train.iloc[val_idx]
            y_tr, y_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]
            
            if name == 'CatBoost':
                model.fit(X_tr, y_tr, eval_set=(X_fold, y_fold), verbose=0)
            else:
                model.fit(X_tr, y_tr)
            
            preds = model.predict(X_fold)
            mae = mean_absolute_error(y_fold, preds)
            fold_mae.append(mae)
            print(f"  Fold {fold+1} MAE: {mae:.4f}")
        
        avg_mae = np.mean(fold_mae)
        mae_scores[name] = avg_mae
        print(f"Average MAE for {name}: {avg_mae:.4f}\n")
    
    return mae_scores

# Function to evaluate Keras Neural Network separately
def evaluate_keras_model(X_train, y_train):
    """
    Evaluate Keras Neural Network using K-Fold Cross-Validation and return MAE scores.
    
    Parameters:
        X_train (pd.DataFrame): Training features.
        y_train (pd.Series): Training target.
    
    Returns:
        dict: Dictionary containing the Keras NN's MAE score.
    """
    kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)
    fold_mae = []
    
    print("Training and evaluating model: Keras Neural Network")
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(X_train)):
        X_tr, X_fold = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_tr, y_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]
        
        # Define the model
        input_dim = X_tr.shape[1]
        model = define_keras_model(input_dim)
        
        # Define early stopping
        early_stop = EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)
        
        # Train the model
        history = model.fit(
            X_tr, y_tr,
            validation_data=(X_fold, y_fold),
            epochs=1000,
            batch_size=32,
            callbacks=[early_stop],
            verbose=0
        )
        
        # Predict and evaluate
        preds = model.predict(X_fold).flatten()
        mae = mean_absolute_error(y_fold, preds)
        fold_mae.append(mae)
        print(f"  Fold {fold+1} MAE: {mae:.4f}")
    
    avg_mae = np.mean(fold_mae)
    print(f"Average MAE for Keras Neural Network: {avg_mae:.4f}\n")
    
    return {'KerasNN': avg_mae}

# Function to train the best model on the entire training data and make predictions
def train_and_predict_best_model(best_model_name, models, X_train, y_train, X_val):
    """
    Retrain the best model on the entire training data and make predictions on the validation set.
    
    Parameters:
        best_model_name (str): Name of the best-performing model.
        models (dict): Dictionary of individual and ensemble models.
        X_train (pd.DataFrame): Training features.
        y_train (pd.Series): Training target.
        X_val (pd.DataFrame): Validation features.
    
    Returns:
        np.ndarray or list: Predictions on the validation set.
    """
    print(f"Retraining the best model: {best_model_name} on the entire training data...")
    
    if best_model_name in models:
        best_model = models[best_model_name]
        if best_model_name == 'CatBoost':
            best_model.fit(X_train, y_train, eval_set=(X_val, y_val), verbose=0)
        else:
            best_model.fit(X_train, y_train)
        predictions = best_model.predict(X_val)
    elif best_model_name in ensemble_models:
        best_model = ensemble_models[best_model_name]
        best_model.fit(X_train, y_train)
        predictions = best_model.predict(X_val)
    elif best_model_name == 'KerasNN':
        # Define and train Keras model on entire training data
        input_dim = X_train.shape[1]
        best_model = define_keras_model(input_dim)
        early_stop = EarlyStopping(monitor='loss', patience=50, restore_best_weights=True)
        best_model.fit(
            X_train, y_train,
            epochs=1000,
            batch_size=32,
            callbacks=[early_stop],
            verbose=0
        )
        predictions = best_model.predict(X_val).flatten()
    else:
        print("Unexpected model selection.")
        sys.exit(1)
    
    # Optional: Clip predictions to a reasonable range if necessary
    predictions = np.clip(predictions, a_min=0, a_max=None)
    
    return predictions

# Main function to execute the comparison
def main():
    # Load and concatenate multiple CSV files
    combined_df = load_and_concatenate_csvs(TRAIN_DATA_DIR)
    
    # Preprocess data
    X_train, X_val, y_train, y_val, ids_train, ids_val, scaler, encoder = preprocess_data(combined_df)
    
    # Define individual models
    models = define_models()
    
    # Define ensemble models
    ensemble_models = define_ensemble_models()
    
    # Combine all models into a single dictionary
    all_models = {**models, **ensemble_models}
    
    # Evaluate individual and ensemble models
    mae_scores = evaluate_models(all_models, X_train, y_train)
    
    # Evaluate Keras Neural Network separately
    keras_mae = evaluate_keras_model(X_train, y_train)
    
    # Combine all MAE scores
    mae_scores.update(keras_mae)
    
    # Display MAE comparison
    print("=== Model Performance Comparison ===")
    for model_name, mae in sorted(mae_scores.items(), key=lambda x: x[1]):
        print(f"{model_name}: MAE = {mae:.4f}")
    
    # Identify the best model
    best_model_name = min(mae_scores, key=mae_scores.get)
    print(f"\nBest performing model: {best_model_name} with MAE = {mae_scores[best_model_name]:.4f}")
    
    # Retrain the best model on the entire training data and make predictions on the validation set
    predictions_val = train_and_predict_best_model(best_model_name, all_models, X_train, y_train, X_val)
    
    # If IDs are present in validation set, prepare submission
    if ids_val is not None:
        submission = pd.DataFrame({
            ID_COLUMN: ids_val,
            TARGET_COLUMN: predictions_val
        })
    else:
        # If no ID column, create a simple index-based submission
        submission = pd.DataFrame({
            'Index': X_val.index,
            TARGET_COLUMN: predictions_val
        })
    
    # Save to CSV with specified encoding and newline characters
    submission.to_csv(SUBMISSION_PATH, index=False, encoding='utf-8', line_terminator='\n')
    
    print(f"Submission file created at: {SUBMISSION_PATH}")
    
    # Optional: Evaluate best model on validation set
    mae = mean_absolute_error(y_val, predictions_val)
    print(f"Validation MAE for the best model ({best_model_name}): {mae:.4f}")

# Entry point
if __name__ == '__main__':
    main()


Loaded file: ./TrainingData\L10_Train.csv with shape (91340, 8)
Loaded file: ./TrainingData\L11_Train.csv with shape (56968, 8)
Loaded file: ./TrainingData\L12_Train.csv with shape (82911, 8)
Loaded file: ./TrainingData\L13_Train.csv with shape (69506, 8)
Loaded file: ./TrainingData\L14_Train.csv with shape (66151, 8)
Loaded file: ./TrainingData\L15_Train.csv with shape (85240, 8)
Loaded file: ./TrainingData\L16_Train.csv with shape (83258, 8)
Loaded file: ./TrainingData\L17_Train.csv with shape (96642, 8)
Loaded file: ./TrainingData\L18_Train.csv with shape (2255, 8)
Loaded file: ./TrainingData\L19_Train.csv with shape (12040, 8)
Loaded file: ./TrainingData\L1_Train.csv with shape (101673, 8)
Loaded file: ./TrainingData\L20_Train.csv with shape (11237, 8)
Loaded file: ./TrainingData\L21_Train.csv with shape (22671, 8)
Loaded file: ./TrainingData\L22_Train.csv with shape (11699, 8)
Loaded file: ./TrainingData\L23_Train.csv with shape (13872, 8)
Loaded file: ./TrainingData\L24_Train.csv

: 

In [2]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from tqdm import tqdm

# ----------------------------
# Configuration and Constants
# ----------------------------

# Define file paths (Update these paths accordingly)
TRAIN_DATA_DIR = './TrainingData'  # Replace with your training data directory path
SUBMISSION_PATH = './submission.csv'  # Replace with your desired submission path

# Define constants
TARGET_COLUMN = 'Power(mW)'  # Replace with your target column name
ID_COLUMN = 'LocationCode'    # Replace with your ID column name (if available)
N_SPLITS = 5                   # Number of folds for cross-validation
RANDOM_STATE = 42

# ----------------------------
# Function Definitions
# ----------------------------

def load_training_data(train_dir):
    """
    Load and concatenate all CSV files from the training directory.
    """
    all_files = [os.path.join(train_dir, f) for f in os.listdir(train_dir) if f.endswith('.csv')]
    if not all_files:
        raise ValueError(f"No CSV files found in the directory: {train_dir}")
    
    df_list = []
    for file in all_files:
        df = pd.read_csv(file)
        df_list.append(df)
    
    combined_df = pd.concat(df_list, ignore_index=True)
    print(f"Loaded {len(all_files)} files with total {combined_df.shape[0]} rows.")
    return combined_df

def preprocess_data(df, target, id_col=None):
    """
    Preprocess the data:
    - Drop ID column if specified
    - Separate features and target
    - Identify numerical and categorical columns
    - Create preprocessing pipeline
    """
    if id_col and id_col in df.columns:
        df = df.drop(columns=[id_col])
    
    X = df.drop(columns=[target])
    y = df[target]
    
    # Identify numerical and categorical columns
    numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
    categorical_cols = X.select_dtypes(include=['object', 'category', 'bool']).columns.tolist()
    
    print(f"Numerical columns: {numerical_cols}")
    print(f"Categorical columns: {categorical_cols}")
    
    # Define preprocessing steps for numerical and categorical data
    numerical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])
    
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])
    
    # Combine preprocessing steps
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, numerical_cols),
            ('cat', categorical_transformer, categorical_cols)
        ]
    )
    
    return X, y, preprocessor

from lightgbm import LGBMRegressor

def get_models():
    """
    Define and return a dictionary of models to compare.
    """
    models = {
        'Linear Regression': LinearRegression(),
        'Random Forest': RandomForestRegressor(
            n_estimators=50,             # Reduced number of trees
            max_depth=10,                # Limited tree depth
            max_features='sqrt',         # Fewer features per split
            min_samples_split=10,        # More samples required to split
            min_samples_leaf=5,          # More samples per leaf
            random_state=RANDOM_STATE,
            n_jobs=-1                    # Utilize all cores
        ),
        'Gradient Boosting': GradientBoostingRegressor(
            n_estimators=100,
            learning_rate=0.1,
            max_depth=3,
            random_state=RANDOM_STATE
        ),
        'LightGBM': LGBMRegressor(
            n_estimators=100,
            learning_rate=0.1,
            num_leaves=31,
            max_depth=10,
            random_state=RANDOM_STATE,
            n_jobs=-1
        ),
        'Decision Tree': DecisionTreeRegressor(
            max_depth=10, 
            min_samples_split=10,
            min_samples_leaf=5,
            random_state=RANDOM_STATE
        )
    }
    return models


def evaluate_model(model, X, y, preprocessor, n_splits=5, random_state=42):
    """
    Evaluate a single model using K-Fold cross-validation and return the total absolute error.
    """
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    total_error = 0
    fold = 1
    
    for train_index, val_index in kf.split(X):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]
        
        # Create a pipeline with preprocessing and the model
        pipeline = Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('model', model)
        ])
        
        # Fit the model
        pipeline.fit(X_train, y_train)
        
        # Predict on validation set
        y_pred = pipeline.predict(X_val)
        
        # Calculate absolute errors
        abs_errors = np.abs(y_pred - y_val)
        
        # Sum the absolute errors as per competition scoring
        fold_error = abs_errors.sum()
        total_error += fold_error
        
        print(f"Fold {fold}: Error = {fold_error}")
        fold += 1
    
    return total_error

def main():
    # Load the training data
    print("Loading training data...")
    df = load_training_data(TRAIN_DATA_DIR)
    
    # Preprocess the data
    print("\nPreprocessing data...")
    X, y, preprocessor = preprocess_data(df, target=TARGET_COLUMN, id_col=ID_COLUMN)
    
    # Get the models to evaluate
    models = get_models()
    
    # Evaluate each model
    print("\nEvaluating models...")
    results = {}
    for model_name, model in tqdm(models.items(), desc="Models"):
        print(f"\nEvaluating {model_name}...")
        total_error = evaluate_model(model, X, y, preprocessor, n_splits=N_SPLITS, random_state=RANDOM_STATE)
        results[model_name] = total_error
        print(f"Total Error for {model_name}: {total_error}")
    
    # Sort results by total error
    sorted_results = sorted(results.items(), key=lambda x: x[1])
    
    # Display the comparison
    print("\nModel Performance Comparison:")
    print("{:<25}{}".format("Model", "Total Absolute Error"))
    print("-" * 40)
    for model_name, error in sorted_results:
        print("{:<25}{:.2f}".format(model_name, error))
    
    # Optionally, save the results to a CSV file
    results_df = pd.DataFrame(sorted_results, columns=['Model', 'Total_Absolute_Error'])
    results_df.to_csv('model_comparison_results.csv', index=False)
    print("\nResults saved to 'model_comparison_results.csv'.")

if __name__ == "__main__":
    main()


Loading training data...
Loaded 24 files with total 1375028 rows.

Preprocessing data...
Numerical columns: ['WindSpeed(m/s)', 'Pressure(hpa)', 'Temperature(°C)', 'Humidity(%)', 'Sunlight(Lux)']
Categorical columns: ['DateTime']

Evaluating models...


Models:   0%|          | 0/5 [00:00<?, ?it/s]


Evaluating Linear Regression...
Fold 1: Error = 38493432.573787495
Fold 2: Error = 38020190.56843455
Fold 3: Error = 38398581.81373827
Fold 4: Error = 38391401.46303085


Models:  20%|██        | 1/5 [00:46<03:04, 46.11s/it]

Fold 5: Error = 38537720.499942146
Total Error for Linear Regression: 191841326.91893333

Evaluating Random Forest...
Fold 1: Error = 93718822.25417185
Fold 2: Error = 91895136.2034235
Fold 3: Error = 93544213.16002515
Fold 4: Error = 93753100.68681169


...

Fold 5: Error = 14601152.008891726
Total Error for Gradient Boosting: 73015247.39533664

Evaluating LightGBM...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.502835 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2759
[LightGBM] [Info] Number of data points in the train set: 1100022, number of used features: 802
[LightGBM] [Info] Start training from score 248.661707
Fold 1: Error = 13240652.759782262
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.529926 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2901
[LightGBM] [Info] Number of data points in the train set: 1100022, number of used features: 872
[LightGBM] [Info] Start training from score 248.967439
Fold 2: Error = 13082798.301974

Models:  80%|████████  | 4/5 [47:06<12:58, 778.36s/it] 

Fold 5: Error = 13115304.849380925
Total Error for LightGBM: 65746489.29028574

Evaluating Decision Tree...
Fold 1: Error = 13386080.44639697
Fold 2: Error = 13284031.69955482
Fold 3: Error = 13276346.722178057
Fold 4: Error = 13365720.340492314


Models: 100%|██████████| 5/5 [50:23<00:00, 604.77s/it]

Fold 5: Error = 13314794.54406684
Total Error for Decision Tree: 66626973.752689004

Model Performance Comparison:
Model                    Total Absolute Error
----------------------------------------
LightGBM                 65746489.29
Decision Tree            66626973.75
Gradient Boosting        73015247.40
Linear Regression        191841326.92
Random Forest            464883307.27

Results saved to 'model_comparison_results.csv'.
